# GeoTIFF
Generating virutal datasets from GeoTiff files

<img src="../images/radar.png" width=400 alt="ARG"></img>

## Overview

In this tutorial we will cover:

1. How to generate virtual datasets from GeoTIFFs.
1. Combining virtual datasets.


## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Basics of virtual Zarr stores](../foundations/01_kerchunk_basics.ipynb) | Required | Core |
| [Multi-file virtual datasets with VirtualiZarr](../foundations/02_kerchunk_multi_file.ipynb) | Required | Core |
| [Parallel virtual dataset creation with VirtualiZarr, Kerchunk, and Dask](../foundations/03_kerchunk_dask) | Required | Core |
| [Introduction to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro) | Required | IO/Visualization |
- **Time to learn**: 30 minutes
---

## About the Dataset

The Finish Meterological Institute (FMI) Weather Radar Dataset is a collection of GeoTIFF files containing multiple radar specific variables, such as rainfall intensity, precipitation accumulation (in 1, 12 and 24 hour increments),  radar reflectivity, radial velocity, rain classification and the cloud top height. It is available through the [AWS public data portal](https://aws.amazon.com/marketplace/pp/prodview-koodet467asui?sr=0-19&ref_=beagle&applicationId=AWSMPContessa) and is updated frequently. 



More details on this dataset can be found [here](https://en.ilmatieteenlaitos.fi/radar-data-on-aws-s3).


In [ ]:
import logging
from datetime import datetime

import dask
import fsspec
import rioxarray
import s3fs
import xarray as xr
from distributed import Client
from virtualizarr import open_virtual_dataset

### Examining a Single GeoTIFF File

Before we use `Kerchunk` to create indices for multiple files, we can load a single GeoTiff file to examine it. 


In [ ]:
# URL pointing to a single GeoTIFF file
url = "s3://fmi-opendata-radar-geotiff/2023/07/01/FIN-ACRR-3067-1KM/202307010100_FIN-ACRR1H-3067-1KM.tif"

# Initialize a s3 filesystem
fs = s3fs.S3FileSystem(anon=True)

xds = rioxarray.open_rasterio(fs.open(url))

In [ ]:
xds

In [ ]:
xds.isel(band=0).where(xds < 2000).plot()

## Create Input File List

Here we are using `fsspec's` glob functionality along with the *`*`* wildcard operator and some string slicing to grab a list of GeoTIFF files from a `s3` `fsspec` filesystem. 

In [ ]:
# Initiate fsspec filesystems for reading
fs_read = fsspec.filesystem("s3", anon=True, skip_instance_cache=True)

files_paths = fs_read.glob(
    "s3://fmi-opendata-radar-geotiff/2023/01/01/FIN-ACRR-3067-1KM/*24H-3067-1KM.tif"
)
# Here we prepend the prefix 's3://', which points to AWS.
files_paths = sorted(["s3://" + f for f in files_paths])

## Start a Dask Client

To parallelize the creation of our reference files, we will use `Dask`. For a detailed guide on how to use Dask and Kerchunk, see the Foundations notebook: [Kerchunk and Dask](../foundations/kerchunk_dask).


In [ ]:
client = Client(n_workers=8, silence_logs=logging.ERROR)
client

In [ ]:
def generate_virtual_dataset(file):
    storage_options = dict(
        anon=True, default_fill_cache=False, default_cache_type="none"
    )
    vds = open_virtual_dataset(
        file,
        indexes={},
        filetype="tiff",
        reader_options={
            "remote_options": {"anon": True},
            "storage_options": storage_options,
        },
    )
    # Pre-process virtual datasets to extract time step information from the filename
    subst = file.split("/")[-1].split(".json")[0].split("_")[0]
    time_val = datetime.strptime(subst, "%Y%m%d%H%M")
    vds = vds.expand_dims(dim={"time": [time_val]})
    # Only include the raw data, not the overviews
    vds = vds[["0"]]
    return vds

In [ ]:
# Generate Dask Delayed objects
tasks = [dask.delayed(generate_virtual_dataset)(file) for file in files_paths]

In [ ]:
# Start parallel processing
import warnings

warnings.filterwarnings("ignore")
virtual_datasets = dask.compute(*tasks)

## Combine virtual datasets

In [ ]:
combined_vds = xr.concat(virtual_datasets, dim="time")
combined_vds

## Shut down the Dask cluster

In [ ]:
client.shutdown()